# Curate results of spike sorting

Notebook within the chronic ephys processing pipeline
- 1-preprocess_acoustics
- 2-curate_acoustics
- 3-sort_spikes
- **4-curate_spikes**

Use the environment **spikeproc** to run this notebook

If necessary, SpikeInterface has a [method](https://spikeinterface.readthedocs.io/en/stable/modules/sorters.html) to concatenate multi-segment recordings 

In [14]:
%matplotlib inline
import pandas as pd
import numpy as np
import os
import pickle
import spikeinterface.full as si

## Set parameters

In [2]:
sess_par = {
    'bird':'z_y19o20_21', # bird ID
    'sess':'2021-10-27', # session date
    'epoch':'1033_undirected_g0-1142_directed_g0', # epoch
    'ephys_software':'sglx', # recording software, sglx or oe
    'sorter':'kilosort3', # spike sorting algorithm
    'sort':'sort_0', # sort index
}

labels = ['sua','mua','noise','1','2','3']

In [2]:
sess_par = {
    'bird':'z_c5o30_23', # bird ID
    'sess':'2023-06-15', # session date
    'epoch':'0913_g0', # epoch
    'ephys_software':'sglx', # recording software, sglx or oe
    'sorter':'kilosort3', # spike sorting algorithm
    'sort':'sort_0', # sort index
}

labels = ['sua','mua','noise','1','2','3']

In [15]:
sess_par = {
    'bird':'z_p5y10_23', # bird ID
    'sess':'2024-05-16', # session date
    'probe':{'probe_type':'neuropixels-2.0'}, # probe specs
    'epoch':'1246_g0', # epoch
    'ephys_software':'sglx', # recording software, sglx or oe
    'sorter':'kilosort4', # spike sorting algorithm
    'sort':'sort_0', # sort index
}

labels = ['sua','mua','noise','1','2','3']

In [16]:
# Trevor's thresholds for automatic unit labeling (don't seem to transfer well to z_c5o30_23)
isi_vr_thresh_low = 0.1
isi_vr_thresh_high = 0.5
snr_thresh_low = 1
snr_thresh_high = 2
labels = []
auto_merge_dict = {
    'minimum_spikes':1000,
    'maximum_distance_um':150,
    'peak_sign':'neg',
    'bin_ms':0.25,
    'window_ms':100,
    'corr_diff_thresh':0.16,
    'template_diff_thresh':0.5, #0.25
    'censored_period_ms':0.3,
    'refractory_period_ms':1,
    'sigma_smooth_ms':0.6,
    'contamination_threshold':0.2,
    'adaptative_window_threshold':0.5,
    'censor_correlograms_ms':0.15,
    'num_channels':5,
    'num_shift':5,
    'firing_contamination_balance':1.5,
    'extra_outputs':False,
    'steps':['min_spikes','remove_contaminated','unit_positions',
             'template_similarity','check_increase_score']
} 
# ['min_spikes','remove_contaminated','unit_positions',
#  'correlogram','template_similarity','check_increase_score']

In [17]:
sort_dir = '/net2/expData/speech_bci/derived_data/{}/{}/{}/{}/{}/{}/'.format(sess_par['bird'],sess_par['sess'],sess_par['ephys_software'],sess_par['epoch'],sess_par['sorter'],sess_par['sort'])
sort_path = sort_dir + 'sorter_output/'
wave_path = sort_dir + 'waveforms/'
metrics_path = wave_path + 'quality_metrics/metrics.csv'
print(sort_dir)

/mnt/cube/chronic_ephys/der/z_p5y10_23/2024-05-16/sglx/1246_g0/kilosort4/sort_0/


In [ ]:
sort = si.read_kilosort(sort_path)
wave = si.load_waveforms(wave_path)

In [6]:
metrics_pd = pd.read_csv(metrics_path)
metrics_list = metrics_pd.keys().tolist()
for this_metric in metrics_list:
    sort.set_property(this_metric, metrics_pd[this_metric].values)

# set thresholds
isi_vr_label = np.full(sort.get_num_units(),'l')
isi_vr_label[np.where((sort.get_property('isi_violations_ratio') > isi_vr_thresh_low) & 
                (sort.get_property('isi_violations_ratio') < isi_vr_thresh_high))[0]] = 'm'
isi_vr_label[np.where(sort.get_property('isi_violations_ratio') > isi_vr_thresh_high)[0]] = 'h'  
sort.set_property('isi_vr_thresh',isi_vr_label)
snr_label = np.full(sort.get_num_units(),'l')
snr_label[np.where((sort.get_property('snr') > snr_thresh_low) & 
                (sort.get_property('snr') < snr_thresh_high))[0]] = 'm'
snr_label[np.where(sort.get_property('snr') > snr_thresh_high)[0]] = 'h' 
sort.set_property('snr_thresh',snr_label)
quality_labels = np.full(sort.get_num_units(),'_____')
quality_labels[np.where(isi_vr_label == 'h')[0]] = 'mua_4'
quality_labels[np.where((isi_vr_label == 'l') & (snr_label == 'h'))[0]] = 'sua_1'
quality_labels[np.where((isi_vr_label == 'l') & (snr_label == 'm'))[0]] = 'sua_2'
quality_labels[np.where((isi_vr_label == 'm') & (snr_label == 'h'))[0]] = 'sua_2'
quality_labels[np.where((isi_vr_label == 'm') & (snr_label == 'm'))[0]] = 'sua_3'
quality_labels[np.where(snr_label == 'l')[0]] = 'noise'
sort.set_property('quality_labels',quality_labels)
print('sua_1:',len(np.where(quality_labels=='sua_1')[0]))
print('sua_2:',len(np.where(quality_labels=='sua_2')[0]))
print('sua_3:',len(np.where(quality_labels=='sua_2')[0]))
print('mua_4:',len(np.where(quality_labels=='mua_4')[0]))
print('noise:',len(np.where(quality_labels=='noise')[0]))
print('total:',len(np.where(quality_labels=='sua_1')[0])+
     len(np.where(quality_labels=='sua_2')[0])+
     len(np.where(quality_labels=='sua_3')[0])+
     len(np.where(quality_labels=='mua_4')[0])+
     len(np.where(quality_labels=='noise')[0]))
wave.sorting = sort
unit_table_properties = ['quality_labels','KSLabel','isi_violations_ratio','snr','num_spikes']
print(sort); print(wave)
# si.plot_probe_map(wave)

sua_1: 56
sua_2: 53
sua_3: 53
mua_4: 228
noise: 0
total: 337
KiloSortSortingExtractor: 337 units - 1 segments - 30.0kHz
WaveformExtractor: 384 channels - 337 units - 1 segments
  before:30 after:60 n_per_units:500 - sparse


In [10]:
# get potential merges
potential_merges = si.get_potential_auto_merge(wave,
    minimum_spikes=auto_merge_dict['minimum_spikes'],
    maximum_distance_um=auto_merge_dict['maximum_distance_um'],
    peak_sign=auto_merge_dict['peak_sign'],
    bin_ms=auto_merge_dict['bin_ms'],
    window_ms=auto_merge_dict['window_ms'],
    corr_diff_thresh=auto_merge_dict['corr_diff_thresh'],
    template_diff_thresh=auto_merge_dict['template_diff_thresh'],
    censored_period_ms=auto_merge_dict['censored_period_ms'],
    refractory_period_ms=auto_merge_dict['refractory_period_ms'],
    sigma_smooth_ms=auto_merge_dict['sigma_smooth_ms'],
    contamination_threshold=auto_merge_dict['contamination_threshold'],
    adaptative_window_threshold=auto_merge_dict['adaptative_window_threshold'],
    censor_correlograms_ms=auto_merge_dict['censor_correlograms_ms'],
    num_channels=auto_merge_dict['num_channels'],
    num_shift=auto_merge_dict['num_shift'],
    firing_contamination_balance=auto_merge_dict['firing_contamination_balance'],
    extra_outputs=auto_merge_dict['extra_outputs'],
    steps=auto_merge_dict['steps'])
print(potential_merges)

[(63, 68)]


## Curate spikes
Using the [SpikeInterface](https://spikeinterface.readthedocs.io/en/latest/) [web-based](https://github.com/SpikeInterface/spikeinterface) [viewer](https://github.com/magland/sortingview), using this [curation guide](https://docs.google.com/document/d/1IUQBDqT2FJ8xv-ZPQRcb4yFb-ptSh8GtmOEt0uaAESc/edit?usp=sharing)

In [ ]:
# skip whatever metrics failed during sort
### remove this
si.plot_quality_metrics(wave, skip_metrics=['amplitude_cutoff'], backend="sortingview");

In [ ]:
si.plot_sorting_summary(waveform_extractor=wave, curation=True, backend='sortingview', label_choices=labels);

## Import curated sorts
When complete, click the button labeled `Save as snapshot (sha1://)` and paste the link below to generate sort_curated, a [sorting](https://spikeinterface.readthedocs.io/en/latest/api.html#spikeinterface.core.BaseSorting) [object](https://github.com/SpikeInterface/spikeinterface/blob/25bb836601e3ec358a06dcb73be34699d8065faf/src/spikeinterface/core/basesorting.py)

In [8]:
sha_uri = 'sha1://72c31c479196f516b28cb2c43ecfd2d8bcd7f555'

In [9]:
%%capture cap
sort_curated = si.apply_sortingview_curation(sorting=sort,uri_or_json=sha_uri,verbose=True)

In [10]:
def find(ch, s):
    return [i for i in range(len(s)) if s[i]==ch]

In [11]:
# get unit IDs
unit_ids = sort_curated.get_unit_ids()
print(f"{len(unit_ids)} units after curation:")

# get merged units
merge_str_all = cap.stdout
merge_starts = find('[',merge_str_all)
merge_stops = find(']',merge_str_all)
merges = [merge_str_all[merge_starts[i]+1:merge_stops[i]].split(',') for i in range(len(merge_starts))]
iui = sort.get_unit_ids() # initial unit IDs
utm = [[int(x) for x in m] for m in merges] # units to merge
nui = np.arange(max(iui)+1, max(iui)+len(utm)+1) # new unit IDs

# set merged properties to unit with highest original spike rate
orig_unit_ids = [[x] for x in unit_ids]
for i, u in enumerate(utm):
    print(f'- Units {u} merged to {nui[i]}')
    idx = [np.where(iui == x)[0][0] for x in u]
    u_n_spks = sort.get_property('num_spikes')[idx]
    max_spikes_i = idx[np.argmax(u_n_spks)]
    nui_i = np.where(unit_ids == nui[i])[0][0]
    for this_metric in metrics_list:
        sort_curated.get_property(this_metric)[nui_i] = sort.get_property(this_metric)[max_spikes_i]
    sort_curated.get_property('num_spikes')[nui_i] = np.sum(u_n_spks)
    orig_unit_ids[nui_i] = u

435 units after curation:
- Units [68, 66] merged to 489
- Units [24, 31] merged to 490
- Units [25, 28, 32] merged to 491
- Units [39, 40] merged to 492
- Units [51, 53, 62] merged to 493
- Units [93, 95, 84] merged to 494
- Units [114, 112, 115, 116, 110] merged to 495
- Units [118, 108] merged to 496
- Units [111, 113] merged to 497
- Units [414, 409, 432, 426] merged to 498
- Units [164, 166, 165, 163] merged to 499


## Create sorted spikes dataframe
Handle sorting object following the how-to guide outlined [here](https://github.com/SpikeInterface/spikeinterface/blob/25bb836601e3ec358a06dcb73be34699d8065faf/doc/how_to/get_started.rst)

In [ ]:
# Probe absolute location (unit locations are relative)
# SI formatting: probe width (x), depth (y), othogonal (z)
# Assuming flat of probe extends M/L, foot is anterior, and vertical implant, use:
# M/L (x), D/V (y), A/P (z) -- units are um
# Typical HVC-RA implant: [2000, 4300, 300]
probe_abs_loc = np.array([2000, 4300, 300])

In [12]:
# make sort dataframe
spk_df = pd.DataFrame({'unit': unit_ids})

# retrieve spike trains
spk_df['spike_train'] = spk_df['unit'].apply(lambda x: sort_curated.get_unit_spike_train(unit_id=x, segment_index=0))

# retrieve unit locations
spk_df['unit_locations'] = list(si.compute_unit_locations(waveform_extractor=wave))

# store absolute unit locations
spk_df['abs_unit_locations'] = spk_df['unit_locations'].apply(lambda x: x+probe_abs_loc)

# store sort properties
for prop in sort_curated.get_property_keys():
    spk_df[prop] = sort_curated.get_property(prop)
spk_df = spk_df.drop(columns=['original_cluster_id'])
spk_df['orig_unit'] = orig_unit_ids
spk_df = spk_df.rename(columns={'sua': 'label_sua',
                                '1': 'label_1',
                                '2': 'label_2',
                                '3': 'label_3',
                                'mua': 'label_mua',
                                'noise': 'label_noise'})

In [21]:
spk_df.tail(1)

unit                                        spike_train  ContamPct  \
434   499  [130, 15199, 15914, 21923, 23492, 24855, 36386...      100.0   

         Amplitude KSLabel KSLabel_repeat  Unnamed: 0  num_spikes  \
434  6.935584e-310     mua            mua         163     53353.0   

     firing_rate  presence_ratio  ...  silhouette  nn_hit_rate  nn_miss_rate  \
434     7.920922             1.0  ...    0.091839       0.5855      0.035969   

     label_mua  label_1  label_sua  label_2  label_noise  label_3  \
434       True     True       True    False         True    False   

                orig_unit  
434  [164, 166, 165, 163]  

[1 rows x 31 columns]

## Save curated spikes

In [20]:
with open(os.path.join(sort_dir, 'spk_df.pickle'), 'wb') as handle:
    pickle.dump(spk_df, handle)

### Visualize spike raster for an example bout

In [4]:
import os
import pickle
import numpy as np
from scipy import signal
from matplotlib import pyplot as plt

In [23]:
# load bout dataframe
bouts_dir = '/mnt/cube/chronic_ephys/der/{}/{}/{}/{}/'.format(sess_par['bird'],sess_par['sess'],sess_par['ephys_software'],sess_par['epoch'])
with open(os.path.join(bouts_dir, 'bout_pd_ap0_curated.pkl'), 'rb') as handle:
    bout_df = pickle.load(handle)

In [53]:
bout_df.index.to_list()

[8, 39, 52, 59, 64, 65, 69, 79, 127, 157, 177, 188]

In [62]:
# retrieve bout info
bout_idx = 52
if bout_df.loc[bout_idx, 'bout_check']:
    waveform = bout_df.loc[bout_idx, 'waveform']
    spectrogram = bout_df.loc[bout_idx, 'spectrogram']
    len_ms = bout_df.loc[bout_idx, 'len_ms']
    start_sample_ap = bout_df.loc[bout_idx, 'start_sample_ap_0']
    fs = sort_curated.get_sampling_frequency()
    print('Bout loaded...')

bout_fs= 40000

Bout loaded...


In [63]:
# retrieve spike trains
def make_raster(spk_df, unit_ids, start_sample, end_sample):
    spk_arr = np.zeros((len(unit_ids), int(end_sample - start_sample)))
    
    for i, unit_id in enumerate(unit_ids):
        unit_spk_t = spk_df.loc[spk_df['unit'] == unit_id, 'spike_train'].iloc[0]
        unit_spk_t = unit_spk_t[(unit_spk_t > start_sample) & (unit_spk_t < end_sample)]
        spk_arr[i, (unit_spk_t - start_sample).astype(int)] = 1
    
    return spk_arr

# make spike raster for plotting
trail_front = 1
trail_end = 1
start_idx = start_sample_ap - (fs * trail_front)
end_idx = start_sample_ap + len_ms/1000*fs + (fs * trail_front)
spike_arr = make_raster(spk_df, spk_df.unit.to_list(), start_idx, end_idx)

# reorder spike train by original unit
orig_unit = spk_df.orig_unit.to_list()
sorted_unit = np.argsort([np.mean(ou) for ou in orig_unit])
spike_arr = spike_arr[sorted_unit]

In [64]:
def make_spec(x, fs, n_window=512, step_ms=1, f_min=100, f_max=9000, cut_off=0.000055):
    msec_samples = int(fs * 0.001)
    n_overlap = n_window - msec_samples * step_ms
    sigma = 1 / 200. * fs
    
    f, t, sxx = signal.spectrogram(x, fs,
                                   nperseg=n_window,
                                   noverlap=n_overlap,
                                   window=signal.gaussian(n_window, sigma),
                                   scaling='spectrum')
    
    if cut_off > 0:
        sxx[sxx < np.max((sxx) * cut_off)] = 1
    
    sxx[f<f_min, :] = 1

    return f[(f>f_min) & (f<f_max)], t, sxx[(f>f_min) & (f<f_max)]

In [65]:
# plot bout with raster
fig, ax = plt.subplots(nrows=3, gridspec_kw={'height_ratios': [1, 3, 20]}, figsize=(10, 15))

# plot sonogram
ax[0].plot(waveform.flatten(), 'black')
ax[0].set_xlim([-trail_front*bout_fs, len(waveform.flatten())+trail_end*bout_fs])
ax[0].set_axis_off()

# plot spectrogram
f, t, sxx = make_spec(waveform.flatten(), fs)
ax[1].pcolormesh(t, f, np.log(sxx), cmap='inferno')
ax[1].set_xlim([-trail_front, t[-1]+trail_end])
ax[1].tick_params(axis='x', labelsize=10)
ax[1].set_ylabel('f (kHz)', fontsize=12)
ax[1].set_yticks([0,2000, 4000, 6000, 8000])
ax[1].set_yticklabels(['0', '2', '4', '6', '8'], fontsize=10)
ax[1].set_ylim([500, 9000])

# plot spike raster
x_trial, x_time = np.where(spike_arr>0)
ax[2].plot(x_time, x_trial, '.', color='k', markersize=0.75, rasterized=True)
ax[2].set_xlim(0, spike_arr.shape[1])
ax[2].set_ylim(-1, spike_arr.shape[0] + 1)
ax[2].set_xticks(np.arange(0, spike_arr.shape[1], fs).tolist())
ax[2].set_xticklabels([str(int(e)) for e in np.arange(0,np.ceil(spike_arr.shape[1]/fs))])
ax[2].set_xlabel('Time (s)', fontsize=12)
ax[2].set_ylabel('Neuron', fontsize=12)
plt.tight_layout()